In [1]:
import tkinter as tk
from tkinter import messagebox
import requests
from PIL import Image, ImageTk
from io import BytesIO

# Replace with your own API keys
WEATHER_API_KEY = '96fa46e1473233212e79ef0c581cb561'
IMAGE_API_KEY = '2aC_Bmo2VnV8J-OO6RlnmJCUwyzA9HPV6G0EZEhqR08'  # Unsplash access key

# Function to get weather data
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to get images of the city
def get_city_images(city):
    url = f"https://api.unsplash.com/search/photos?query={city}&client_id={IMAGE_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return [result['urls']['regular'] for result in data['results']]
    
    return []  # Return an empty list if no images found

# Function to display weather and city images
def display_weather():
    city = app.city_entry.get()
    weather_data = get_weather(city)
    
    if weather_data:
        city_name = weather_data['name']
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        wind_speed = weather_data['wind']['speed']
        weather_description = weather_data['weather'][0]['description']
        
        weather_info = (f"Weather in {city_name}:\n"
                        f"Temperature: {temperature}°C\n"
                        f"Humidity: {humidity}%\n"
                        f"Wind Speed: {wind_speed} m/s\n"
                        f"Description: {weather_description.capitalize()}")

        app.weather_label.config(text=weather_info)
        
        city_images = get_city_images(city)
        if city_images:
            app.images = city_images  # Store the images in the app
            app.current_image_index = 0  # Start at the first image
            show_image(app.images[app.current_image_index])
        else:
            app.image_label.config(image='')
            messagebox.showwarning("Warning", "No images found for the city.")
    else:
        messagebox.showerror("Error", "City not found.")

def show_image(image_url):
    image_response = requests.get(image_url)
    img_data = Image.open(BytesIO(image_response.content))
    img = img_data.resize((400, 300), Image.LANCZOS)
    img_tk = ImageTk.PhotoImage(img)
    app.image_label.config(image=img_tk)
    app.image_label.image = img_tk

def next_image():
    if hasattr(app, 'images') and app.current_image_index < len(app.images) - 1:
        app.current_image_index += 1
        show_image(app.images[app.current_image_index])

def previous_image():
    if hasattr(app, 'images') and app.current_image_index > 0:
        app.current_image_index -= 1
        show_image(app.images[app.current_image_index])

# GUI application
class WeatherApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Weather Predictor")

        tk.Label(master, text="Enter City Name:").pack(pady=10)
        self.city_entry = tk.Entry(master)
        self.city_entry.pack(pady=5)

        tk.Button(master, text="Get Weather", command=display_weather).pack(pady=20)

        self.weather_label = tk.Label(master, text="", justify="center")
        self.weather_label.pack(pady=10)

        self.image_label = tk.Label(master)
        self.image_label.pack(pady=10)

        button_frame = tk.Frame(master)
        button_frame.pack(pady=10)

        tk.Button(button_frame, text="Previous", command=previous_image).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="Next", command=next_image).pack(side=tk.RIGHT, padx=5)

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = WeatherApp(root)
    root.mainloop()
